In [1]:
import os
from pathlib import Path
from whitebox import WhiteboxTools

wbt = WhiteboxTools()


### Settings 

In [ ]:
WORKING_DIR = Path(r'E:\MACS_Batch_Processing\WA_SelawikSlump_01_20210701_10cm\07_experiments\point_cloud')
RESOLUTION=0.1
TRIANGLE_EDGE_LENGTH=3
fill_FILTER=int(5/RESOLUTION)
POINT_CLOUDS='nir' #('nir', 'rgb', 'both')

In [12]:
wbt.set_working_dir(WORKING_DIR)
#wbt.set_compress_rasters(True)
wbt.set_compress_rasters(True)
def my_callback(value):
    if not "%" in value:
        print(value)
wbt.set_default_callback(my_callback)

### Experiment 
1. merge
2. Interpolate to DSM
3. Smooth output
4. fill gaps

#### Merge 

In [ ]:
regex_filter = dict{'nir':'Grayscale', 'rgb':'group1', 'both':''}

In [19]:
#regex='*.las'
regex=f'*{regex_filter[POINT_CLOUDS]}*.las'
merge_list = list(WORKING_DIR.glob(regex))

merged_pc=f'merged_{POINT_CLOUDS}.las'
inputs = ','.join([f.name for f in merge_list if f.name!=merged_pc])

wbt.lidar_join(
    inputs=inputs,
    output=merged_pc
)

.\whitebox_tools.exe --run="LidarJoin" --wd="E:\MACS_Batch_Processing\WA_SelawikSlump_01_20210701_10cm\07_experiments\point_cloud" --inputs='WA_SelawikSlump_01_20210701_10cm_Grayscale_densified_point_cloud_part_1.las,WA_SelawikSlump_01_20210701_10cm_Grayscale_densified_point_cloud_part_2.las,WA_SelawikSlump_01_20210701_10cm_Grayscale_densified_point_cloud_part_3.las,WA_SelawikSlump_01_20210701_10cm_Grayscale_densified_point_cloud_part_4.las' --output='merged.las' -v --compress_rasters=True

****************************
* Welcome to LidarJoin     *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Adding file: 1 of 4
Adding file: 2 of 4
Adding file: 3 of 4
Adding file: 4 of 4
Writing output LAS file...
Complete!


0

#### DSM 

In [20]:
infile = merged_pc
outfile = f'merged_dsm_RES-{RESOLUTION}_TEL-{TRIANGLE_EDGE_LENGTH}.tif'
wbt.lidar_digital_surface_model(
    i=infile, 
    output=outfile, 
    resolution=RESOLUTION, 
    radius=0.5,
    max_triangle_edge_length=TRIANGLE_EDGE_LENGTH
)

.\whitebox_tools.exe --run="LidarDigitalSurfaceModel" --wd="E:\MACS_Batch_Processing\WA_SelawikSlump_01_20210701_10cm\07_experiments\point_cloud" --input='merged.las' --output='merged_dsm_RES-0.1_TEL-3.tif' --resolution=0.1 --radius=0.5 --max_triangle_edge_length='3' -v --compress_rasters=True

***************************************
* Welcome to LidarDigitalSurfaceModel *
* Powered by WhiteboxTools            *
* www.whiteboxgeo.com                 *
***************************************
Performing interpolation...
Reading input LiDAR file...
Performing triangulation...
Saving data...
Finished merged (1 of 1)
Elapsed Time (including I/O): 5min 34.548s


0

#### Fill Holes

In [21]:
output_filled = outfile[:-4]+f'_filled-{fill_FILTER}.tif'
wbt.fill_missing_data(
    i=outfile, 
    output=output_filled, 
    filter=fill_FILTER, 
    weight=2.0, 
    no_edges=True
)

.\whitebox_tools.exe --run="FillMissingData" --wd="E:\MACS_Batch_Processing\WA_SelawikSlump_01_20210701_10cm\07_experiments\point_cloud" --input='merged_dsm_RES-0.1_TEL-3.tif' --output='merged_dsm_RES-0.1_TEL-3_filled-50.tif' --filter=50 --weight=2.0 --no_edges -v --compress_rasters=True

******************************
* Welcome to FillMissingData *
* Powered by WhiteboxTools   *
* www.whiteboxgeo.com        *
******************************
Reading data...
Finding edge-connected NoData values...
Interpolating data holes...
Saving data...
Output file written
Elapsed Time (excluding I/O): 1min 7.672s


0

#### Filter DSM

In [ ]:
smF=int(10/RESOLUTION)
for iterations in [20]:
    for normdiff in [100]:
        output_filled_smoothed = output_filled[:-4] + f'_smoothed_smF-{smF}_iterations-{iterations}_normDiff-{normdiff}.tif'
        output_filled_smoothed_hs = output_filled_smoothed[:-4] + '_hs.tif'
        wbt.feature_preserving_smoothing(
            output_filled, 
            output_filled_smoothed, 
            filter=smF, 
            norm_diff=normdiff, 
            num_iter=iterations, 
            max_diff=0.5, 
            zfactor=None
        ) 
        wbt.hillshade(dem=output_filled_smoothed,output=output_filled_smoothed_hs)
            

.\whitebox_tools.exe --run="FeaturePreservingSmoothing" --wd="E:\MACS_Batch_Processing\WA_SelawikSlump_01_20210701_10cm\07_experiments\point_cloud" --dem='merged_dsm_RES-0.1_TEL-3_filled-50.tif' --output='merged_dsm_RES-0.1_TEL-3_filled-50_smoothed_smF-100_iterations-20_normDiff-100.tif' --filter=100 --norm_diff=100 --num_iter=20 --max_diff=0.5 -v --compress_rasters=True

*****************************************
* Welcome to FeaturePreservingSmoothing *
* Powered by WhiteboxTools              *
* www.whiteboxgeo.com                   *
*****************************************
Reading data...
Calculating normal vectors: 3.680s
Smoothing normal vectors: 19min 48.444s
Updating elevations...
Iteration 1 of 20...
Iteration 2 of 20...
Iteration 3 of 20...
Iteration 4 of 20...


#### Clip to tiles 
* Use either 
1. footprints file
or 
2. extract footprints of Ortho

#### GDAL retile
https://gdal.org/programs/gdal_retile.html

In [17]:
print('')